In [2]:
import cv2
import numpy as np
from utils.object_detector import ObjectDetector
from utils.template_matcher import TemplateMatcher

In [3]:
detector = ObjectDetector()
matcher = TemplateMatcher()

In [4]:
if "google.colab" in str(get_ipython()):
    from google.colab.patches import cv2_imshow

    imshow = cv2_imshow
else:

    def imshow(img):
        cv2.imshow("ImageWindow", img)
        cv2.waitKey()

        cv2.destroyAllWindows()


In [5]:
img1  = cv2.imread('resources/img1.jpg')
img2  = cv2.imread('resources/img4.jpg')
img1 = cv2.resize(img1, None, fx=0.2, fy=0.2)
img2 = cv2.resize(img2, None, fx=0.2, fy=0.2)

In [6]:
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

## Helper functions

## Detecting objects on a photo

In [7]:
detected_objects, detected_cords = detector.detect_objects(img2)

In [8]:
for i, obj in enumerate(detected_objects):
  corners = detector.detect_corners(obj)
  if len(corners > 0):
    transformed_image = detector.perspective_transform(obj, corners)
    detected_objects[i] = transformed_image

### Idea is to create templates for matching different objects, so far:
 - Capitol
 - Any non reversed card (template for a card is just a random card)

In [9]:
# classes = ['Capitol', 'Opponent Capitol', 'Unit', 'Opponent Unit', 'Support', 'Opponent Support', 'Deck', 'Opponent Deck']
classes = ['Capitol', 'Unit', 'Support', 'Deck']

In [10]:
capitol= cv2.imread('resources/capitol_template.png')
capitol_opp = cv2.imread('resources/capitol_opp_template.png')
unit = cv2.imread('resources/unit_template.png')
unit_opp = cv2.imread('resources/unit_opp_template.png')
support = cv2.imread('resources/support_template.png')
support_opp = cv2.imread('resources/support_opp_template.png')
deck = cv2.imread('resources/deck_template.png')
deck_opp = cv2.imread('resources/deck_opp_template.png')

templates = [capitol, capitol_opp, unit, unit_opp, support, support_opp, deck, deck_opp]
# templates = [capitol, unit, support, deck]
# for template in templates:
#   imshow(template)

### Used templates:
![Capitol template](resources/capitol_template.png)
![Unit template](resources/unit_template.png)
![Support template](resources/support_template.png)
![Deck template](resources/deck_template.png)

## Matching templates

Templates are scaled to size of detected object and then each template is matched to each object to return which objects match given template, instead of hard coded value we would like to explore determining objects by most simmilar template given some small threshold is passed to exclude noise

In [11]:
for obj in detected_objects:
  imshow(obj)

In [12]:
determined_classes = matcher.match_templates(templates, detected_objects)

In [13]:
imshow(matcher.draw_detections(img2, detected_cords, determined_classes, classes))

In [15]:
video_path = 'resources/vid3.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Couldn't open video.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_path = 'outputs/video.avi'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_number = 0
process_every_n_frames = 20  # Process every 5th frame
deck_detected = False
while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_number % process_every_n_frames == 0:
        detected_objects, detected_cords = detector.detect_objects(frame)
        for obj in detected_objects:
           corners = detector.detect_corners(obj)
           if len(corners > 0):
                transformed_image = detector.perspective_transform(obj, corners)
                detected_objects[i] = transformed_image
        determined_classes = matcher.match_templates(templates, detected_objects)
        detected_this_frame = False
        for obj_class in determined_classes:
          if np.argmax(obj_class) == 6 and obj_class[np.argmax(obj_class)] > 0.2:
            deck_detected = True
            detected_this_frame = True
            break
        if not detected_this_frame:
          deck_detected = False
    if not deck_detected:
       cv2.putText(frame, "Card Drawn!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    frame = matcher.draw_detections(frame, detected_cords, determined_classes, classes)

    out.write(frame)

    frame_number += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break


cap.release()
out.release()

cv2.destroyAllWindows()
